## <center> Scraping user for job <center>

- Content: Build a bot to collect user information with the desired job name and export it to a .csv file

#### <center>1. Import library<center>

In [56]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from time import sleep 
from bs4 import BeautifulSoup # for parsing html
import csv # for writing csv file


- open the LinkedIn login page

In [57]:
driver = webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(2)

#### <center>2. Login Linkedin<center>

- Avoid situations where account information is displayed

In [58]:
credential = open('user_pass.txt')
line = credential.readlines()
username = line[0]
password = line[1]

In [59]:
# Find the email and password fields
email_field = driver.find_element(By.ID, 'username')
# Send the keys
email_field.send_keys(username)
sleep(2)

In [60]:
# Find the password field
pass_field = driver.find_element(By.ID, 'password')
# Send the keys
pass_field.send_keys(password)
sleep(2)

- Click button Login

In [61]:
# Find the login button
login_field = driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button')
# Click the login button
linkedin_login = login_field.click()
sleep(2)

#### <center>3. Find users with desired jobs<center>

- Enter the job name which you want to find

In [62]:
# Find the search field
search_field = driver.find_element(By.XPATH,'//*[@id="global-nav-typeahead"]/input')
# Send the keys
search_jobs = input('Enter the job title: ')
search_field.send_keys(search_jobs + ' people')
# Click the search button
search_field.send_keys(Keys.RETURN)
sleep(1)


- Navigate to the all people of job name which you are entered

In [63]:
wait = WebDriverWait(driver, 10) # wait for 10 seconds
search_people_field = wait.until(EC.presence_of_element_located((By.XPATH, "//*[text()='People']"))) # wait until the element is located

all_people_field = search_people_field.click() # click the element

- Select URL users of one page linkedin

In [64]:
def getURL():
    page = BeautifulSoup(driver.page_source)
    profiles = page.find_all('a', class_='app-aware-link')
    all_profiles = []
    for profile in profiles:
        profile_url = profile.get('href')
        if profile_url not in all_profiles:
            if 'https://www.linkedin.com/in/' in profile_url:
                all_profiles.append(profile['href'])
    return all_profiles


- Go to the next page and select the user URL whose number you entered

In [65]:
# number_of_pages = int(input('Enter the number of pages you want to scrape: '))
number_of_user = int(input('Enter the number of users you want to scrape: '))
Url_all = []
# for page in range(number_of_pages):
while len(Url_all) < number_of_user:
    URLs_onepage = getURL()
    sleep(2)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # Scroll down to the bottom of the page
    sleep(1)
    next_button = driver.find_element(By.CLASS_NAME, 'artdeco-pagination__button--next')
    next_button.click()# Click the next button
    Url_all = Url_all + URLs_onepage
    sleep(2)

#### <center> 4. Export file .cvs <center>

In [66]:
file_name = 'output_' +  str(search_jobs)
with open(file_name + '.csv', 'w', newline='', encoding='utf-8') as f:
    headers = ['Name user', 'Address User', 'Role User', 'URL']
    writer = csv.DictWriter(f,delimiter=',',lineterminator='\n', fieldnames=headers)
    writer.writeheader()
    
    for url in Url_all:
        driver.get(url)
        sleep(2)
        page_source =driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        user_infor = soup.find('div', class_='mt2 relative')
        user_name = user_infor.find('h1').text.strip()
        user_address = user_infor.find('span',class_='text-body-small inline t-black--light break-words').text.strip()
        user_role = user_infor.find('div',class_='text-body-medium break-words').text.strip()
        print(f'user_name:  {user_name},\nuser_address: {user_address},\nuser_role: {user_role} \n\n')
        writer.writerow({headers[0]:user_name, headers[1]: user_address, headers[2]: user_role, headers[3]: url})
    print("Export succesfull!")

user_name:  Lucero Chargoy Juarez,
user_address: Barcelona, Catalonia, Spain,
user_role: Tokenomics | DeFi | DataScience🐍| Startup 🚀| I+D+i🇲🇽🇪🇸 


user_name:  Karina Osartchuk,
user_address: Boca Raton, Florida, United States,
user_role: Disruptive Management & Human Capital <3 | CoCreate CEO &Innovative Chairwoman - Brasil & USA | Teams Brasil | ScottMaddenConsulting #gbs #speaker #change #innovate #datascience #people #neuroscience 


user_name:  Karthik Josyula,
user_address: Brookfield, Wisconsin, United States,
user_role: Global Manager | Data Science @ KOHLER | Strategy & Delivery | Leading people and Teaching machines 


user_name:  Nicholas Carter,
user_address: Glen Burnie, Maryland, United States,
user_role: Training Specialist I People-centered | Creating Experiential Learning Opportunities 


user_name:  Eulan Oliveira dos Santos,
user_address: Boadilla del Monte, Community of Madrid, Spain,
user_role: Culture l BlackBelt | Efficiency | Finance | Analytics | SAP ERP & Procu